In [42]:
#First we upload our packages
import arcpy
from arcpy import env  
from arcpy.sa import *
import pandas as pd
import geopandas as gpd
import numpy as np
import random
import psycopg2
from shapely.geometry import Point, Polygon

In [37]:
#Next is to read in the CSV of our September 2023 data as dataframe
csv_file_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\temp_data.csv"
temp_data = pd.read_csv(csv_file_path)

In [38]:
# Then we must convert numeric columns to appropriate data types
numeric_cols = ['high_F', 'low_F', 'precip', 'snow_inch', 'snowd_inch']
temp_data[numeric_cols] = temp_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [39]:
#This data also needs coordinate info for each point
#We will use a Mesonet URL to gather reporting station location
url = "https://mesonet.agron.iastate.edu/sites/networks.php?network=MN_COOP&format=csv&nohtml=on"

# Read the CSV data from the URL into a data frame
loc_df = pd.read_csv(url)

In [40]:
# Merge the "lat" and "lon" columns from df based on the matching "nwsli" and "stid" columns
temp_data = pd.merge(temp_data, loc_df[['stid', 'lat', 'lon']], how='left', left_on='nwsli', right_on='stid')

# Drop the redundant "stid" column
temp_data.drop(columns=['stid'], inplace=True)

# Display the updated precip_data data frame
print(temp_data)

      nwsli        date      time  ...  snowd_inch       lat       lon
0     MOOM5  2023-09-01       NaN  ...         NaN  46.45000 -92.75780
1     TWRM5  2023-09-01       NaN  ...         NaN  46.96670 -95.66670
2     HKHM5  2023-09-01   7:00 AM  ...         NaN  43.76372 -91.34814
3     BABM5  2023-09-01       NaN  ...         NaN  47.71210 -91.95330
4     NHPM5  2023-09-01  11:00 PM  ...         0.0  45.01000 -93.37920
...     ...         ...       ...  ...         ...       ...       ...
5695  NHPM5  2023-09-30  11:00 PM  ...         0.0  45.01000 -93.37920
5696  PKGM5  2023-09-30   8:00 AM  ...         0.0  47.25000 -93.59000
5697  PELM5  2023-09-30   8:00 AM  ...         0.0  46.58330 -96.08890
5698  LCHM5  2023-09-30   8:00 AM  ...         0.0  45.12790 -94.53480
5699  LMBM5  2023-09-30   7:00 AM  ...         0.0  44.24000 -95.32000

[5700 rows x 10 columns]


In [43]:
# Define Minnesota boundary box
minnesota_boundary = Polygon([( -97.5, 43.0), (-89.0, 43.0), (-89.0, 49.5), (-97.5, 49.5)])

# Check if the data falls within the Minnesota boundary
temp_data['Coordinates'] = list(zip(temp_data.lon, temp_data.lat))
temp_data['Coordinates'] = temp_data['Coordinates'].apply(Point)
gdf = gpd.GeoDataFrame(temp_data, geometry='Coordinates')

within_minnesota = gdf[gdf.geometry.within(minnesota_boundary)]
if len(within_minnesota) == 0:
    print("No data falls within Minnesota boundary.")
else:
    print("Data falls within Minnesota boundary.")

Data falls within Minnesota boundary.


In [8]:
# Convert the "Date" column to datetime type
temp_data['date'] = pd.to_datetime(temp_data['date'])

# Check the type of the "Date" column after conversion
date_column_type = temp_data['date'].dtype

print("Type of 'date' column after conversion:", date_column_type)

Type of 'date' column after conversion: datetime64[ns]


In [9]:
#Now we need to choose the date which we will interpolate for
input_date = input("Enter a date (YYYY-MM-DD format): ")

# Convert input string to datetime object
input_date = pd.to_datetime(input_date)

# Filter DataFrame based on the input date
temp_data = temp_data[temp_data['date'] == input_date]

# Display the filtered DataFrame
print(temp_data)

Enter a date (YYYY-MM-DD format): 2023-09-16
      nwsli       date      time  ...  snowd_inch      lat      lon
2850  NISM5 2023-09-16       NaN  ...         NaN  46.5000 -94.2667
2851  CRLM5 2023-09-16   8:00 AM  ...         0.0  46.6700 -94.1100
2852  BWNM5 2023-09-16   8:00 AM  ...         0.0  44.7335 -94.3417
2853  WRUM5 2023-09-16  10:00 PM  ...         NaN  48.9008 -95.4006
2854  RUDM5 2023-09-16   7:00 AM  ...         0.0  43.8052 -91.7501
...     ...        ...       ...  ...         ...      ...      ...
3035  LSAM5 2023-09-16       NaN  ...         NaN  44.9783 -93.2469
3036  ALXM5 2023-09-16   6:00 AM  ...         NaN  45.8782 -95.3827
3037  BLHM5 2023-09-16       NaN  ...         NaN  45.8608 -94.3600
3038  KIMM5 2023-09-16   6:00 AM  ...         0.0  45.3533 -94.3056
3039  MPXM5 2023-09-16  11:00 PM  ...         0.0  44.8496 -93.5644

[190 rows x 10 columns]


In [10]:
# This cell will calculate average temperature for each row
temp_data['avg_temp'] = (temp_data['high_F'] + temp_data['low_F']) / 2

,nwsli,date,time,high_F,low_F,precip,snow_inch,snowd_inch,lat,lon,avg_temp
2850,NISM5,2023-09-16,NaN,NaN,NaN,NaN,NaN,NaN,46.5000,-94.2667,NaN
2851,CRLM5,2023-09-16,8:00 AM,71.0,50.0,0.00,0.0,0.0,46.6700,-94.1100,60.5
2852,BWNM5,2023-09-16,8:00 AM,75.0,51.0,0.00,0.0,0.0,44.7335,-94.3417,63.0
2853,WRUM5,2023-09-16,10:00 PM,NaN,NaN,NaN,NaN,NaN,48.9008,-95.4006,NaN
2854,RUDM5,2023-09-16,7:00 AM,67.0,53.0,0.05,0.0,0.0,43.8052,-91.7501,60.0
...,...,...,...,...,...,...,...,...,...,...,...
3035,LSAM5,2023-09-16,NaN,68.0,54.0,0.08,0.0,NaN,44.9783,-93.2469,61.0
3036,ALXM5,2023-09-16,6:00 AM,NaN,NaN,0.00,0.0,NaN,45.8782,-95.3827,NaN
3037,BLHM5,2023-09-16,NaN,NaN,NaN,NaN,NaN,NaN,45.8608,-94.3600,NaN
3038,KIMM5,2023-09-16,6:00 AM,76.0,50.0,0.00,0.0,0.0,45.3533,-94.3056,63.0


In [11]:
# Then these next cells will calculate evapotranspiration
# First it defines the constant for the Hargreaves method
constant = 0.0023

# Convert 'date' column to datetime object
temp_data['date'] = pd.to_datetime(temp_data['date'])

In [12]:
# Then this cell calculates ET using the Hargreaves method for each row
ET0_values = []
for index, row in temp_data.iterrows():
    avg_temperature = row['avg_temp']
    temperature_range = row['high_F'] - row['low_F']
    
    # Calculate the day of the year
    day_of_year = row['date'].dayofyear
    
    # Calculate ET0 using the Hargreaves method
    ET0 = constant * (avg_temperature + 17.8) * (temperature_range ** 0.5) * (1.0 + 0.033 * np.sin(np.deg2rad(360 * (day_of_year - 81) / 365)))
    
    ET0_values.append(ET0)

# Add the calculated ET0 values as a new column to the DataFrame
temp_data['ET'] = ET0_values

# Display the updated DataFrame
print(temp_data)


      nwsli       date      time  high_F  ...      lat      lon  avg_temp        ET
2850  NISM5 2023-09-16       NaN     NaN  ...  46.5000 -94.2667       NaN       NaN
2851  CRLM5 2023-09-16   8:00 AM    71.0  ...  46.6700 -94.1100      60.5  0.827384
2852  BWNM5 2023-09-16   8:00 AM    75.0  ...  44.7335 -94.3417      63.0  0.912751
2853  WRUM5 2023-09-16  10:00 PM     NaN  ...  48.9008 -95.4006       NaN       NaN
2854  RUDM5 2023-09-16   7:00 AM    67.0  ...  43.8052 -91.7501      60.0  0.671242
...     ...        ...       ...     ...  ...      ...      ...       ...       ...
3035  LSAM5 2023-09-16       NaN    68.0  ...  44.9783 -93.2469      61.0  0.679870
3036  ALXM5 2023-09-16   6:00 AM     NaN  ...  45.8782 -95.3827       NaN       NaN
3037  BLHM5 2023-09-16       NaN     NaN  ...  45.8608 -94.3600       NaN       NaN
3038  KIMM5 2023-09-16   6:00 AM    76.0  ...  45.3533 -94.3056      63.0  0.950022
3039  MPXM5 2023-09-16  11:00 PM    70.0  ...  44.8496 -93.5644      58.5  0

In [14]:
#We will determine how many null values lie in our data
# This line will calculate the total number of rows
total_rows = temp_data.shape[0]

# Calculate the number of null values in avg_temp and ET columns
null_avg_temp = temp_data['avg_temp'].isnull().sum()
null_ET = temp_data['ET'].isnull().sum()

# Calculate the percentage of null values
null_avg_temp_percentage = (null_avg_temp / total_rows) * 100
null_ET_percentage = (null_ET / total_rows) * 100

# Define the threshold for warning
threshold = 50

# Finally, this line will let us know if the percentage of null values exceeds the threshold for either column
if null_avg_temp_percentage > threshold or null_ET_percentage > threshold:
    print("Warning: The percentage of null values in either avg_temp or ET column exceeds 50%.")


In [15]:
# Then we need to create a feature class for our points
output_fc = 'Points'

# Create a new feature class
arcpy.management.CreateFeatureclass(
    arcpy.env.workspace,
    output_fc,
    'POINT',
    spatial_reference=arcpy.SpatialReference(4326)  # WGS84 Geographic Coordinate System
)

# Check the data type of the 'ET' column in the DataFrame
et_dtype = temp_data['ET'].dtype

# Add field to store ET data, ensuring correct data type
if et_dtype == 'float64':
    arcpy.management.AddField(output_fc, 'ET', 'FLOAT')
else:
    arcpy.management.AddField(output_fc, 'ET', 'DOUBLE')

# Add 'date' and 'nwsli' fields
arcpy.management.AddField(output_fc, 'date', 'DATE')
arcpy.management.AddField(output_fc, 'nwsli', 'TEXT')

# Add 'GDD' field
arcpy.management.AddField(output_fc, 'GDD', 'FLOAT')

# Open an insert cursor
with arcpy.da.InsertCursor(output_fc, ['SHAPE@XY', 'ET', 'date', 'nwsli', 'GDD']) as cursor:
    # Iterate over each row in the DataFrame
    for index, row in temp_data.iterrows():
        # Extract lat, lon, ET, GDD, date, and nwsli values
        lat = row['lat']
        lon = row['lon']
        date = row['date']
        ET = row['ET']
        GDD = row['GDD']  # Fetch GDD value
        nwsli = row['nwsli']
        
        # Create a point geometry
        point = arcpy.Point(lon, lat)
        point_geometry = arcpy.PointGeometry(point)
        
        # Insert the point feature with the ET, GDD, date, and nwsli values
        cursor.insertRow([point_geometry, ET, date, nwsli, GDD])

print(f"Feature class '{output_fc}' created successfully.")


Feature class 'Points' created successfully.


In [16]:
# It is the intention to use IDW for interpolation, but we will set aside sample points to check our interpolations later
# In order to test our interpolations later, we will need to create a subset of this data and remove it from the feature layer
input_feature_layer = "Points"

# We will save it as a separate feature layer
output_feature_layer = "Random_Selected_Points"
output_feature_class = "Random_Selected_Points"

# We first need to get the total count of features in the input feature layer
total_features_count = int(arcpy.GetCount_management(input_feature_layer).getOutput(0))

# The I will generate a list of 36 random indices
random_indices = random.sample(range(1, total_features_count + 1), 36)

# I will use a SQL expression to select the randomly chosen features
sql_expression = "OBJECTID IN ({})".format(','.join(map(str, random_indices)))

# Then I will create a new feature layer with the randomly selected features
arcpy.MakeFeatureLayer_management(input_feature_layer, output_feature_layer, sql_expression)

print("Randomly selected 36 features and created a new feature layer:", output_feature_layer)

# The I will save the selected features to a new feature class
arcpy.CopyFeatures_management(output_feature_layer, output_feature_class)
print("Saved the selected features to a new feature class:", output_feature_class)


Randomly selected 36 features and created a new feature layer: Random_Selected_Points
Saved the selected features to a new feature class: Random_Selected_Points


In [17]:
#This performs the IDW interpolation
outIDW = Idw("Points.shp", "ET")
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\IDW_ETPoints.tif"
outIDW.save(output_path)

In [18]:
#This performs the kriging interpolation
outKriging = Kriging("Points.shp", "ET", KrigingModelOrdinary("CIRCULAR", 2000, 2.6, 542, 0))
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\Kriging_ETPoints.tif"
outKriging.save(output_path)

In [19]:
#This performs the resampling of the IDW tif to reduce the number of points needed
arcpy.management.Resample(
    in_raster=r"Idw_ETPoints.tif",
    out_raster=r"Idw_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

arcpy.management.Resample(
    in_raster=r"Kriging_ETPoints.tif",
    out_raster=r"Kriging_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Kriging_ET_Resample'>

In [20]:
#This converts the raster to points
arcpy.conversion.RasterToPoint(
    in_raster=r"Idw_ET_Resample",
    out_point_features=r"Idw_ET_Point",
    raster_field="value"
)

arcpy.conversion.RasterToPoint(
    in_raster=r"Kriging_ET_Resample",
    out_point_features=r"Kriging_ET_Point",
    raster_field="value"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Kriging_ET_Point'>

In [21]:
#To test the model, this samples the data
arcpy.sa.Sample(
    in_rasters="Idw_ETPoints.tif",
    in_location_data="Random_Selected_Points",
    out_table=r"Sample_Idw_ET",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [22]:
arcpy.sa.Sample(
    in_rasters="Kriging_ETPoints.tif",
    in_location_data="Random_Selected_Points",
    out_table=r"Sample_Kriging_ET",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [23]:
# Next we will create a dataframe based on our randomly selected features from earlier
input_feature_class = "Random_Selected_Points"

# We will first convert feature classes to a NumPy array
fields = ["OID@","nwsli", "ET"]
array = arcpy.da.FeatureClassToNumPyArray(input_feature_class, fields)

# Then convert the NumPy array to a dataframe
df_ET = pd.DataFrame(array)

# Rename the OID@ field to ObjectID
df_ET.rename(columns={"OID@": "OBJECTID"}, inplace=True)

# Then we will print the DataFrame to check it
print(df_ET)


    OBJECTID  nwsli        ET
0          1  BWNM5  0.912751
1          2  WDOM5  0.852262
2          3  ARTM5  0.879691
3          4  RWGM5       NaN
4          5  ORVM5       NaN
5          6  LUVM5       NaN
6          7  SPGM5       NaN
7          8  ALBM5  0.621449
8          9  WNNM5  0.791975
9         10  THLM5  0.681406
10        11  LMBM5  0.973683
11        12  LCHM5  1.037180
12        13  AITM5       NaN
13        14  MRAM5  1.023934
14        15  MADM5  1.007857
15        16  OWAM5  0.671242
16        17  LKFM5  0.816817
17        18  LEIM5  0.725412
18        19  STIM5  0.717588
19        20  WAAM5  0.740415
20        21  TETM5       NaN
21        22  AGGM5  0.827264
22        23  COTM5       NaN
23        24  CRNM5  0.797050
24        25  THRM5  0.681406
25        26  BRVM5  0.961481
26        27  WRIM5  0.912675
27        28  ASNM5  0.817152
28        29  MMLM5  0.998086
29        30  HLLM5  0.613462
30        31  MWDM5       NaN
31        32  TOHM5  0.837950
32        

In [24]:
# Next we will create a dataframe based on our randomly selected features from earlier
input_feature_class = "Random_Selected_Points"

# We will first convert feature classes to a NumPy array
fields = ["OID@","nwsli", "GDD"]
array = arcpy.da.FeatureClassToNumPyArray(input_feature_class, fields)

# Then convert the NumPy array to a dataframe
df_GDD = pd.DataFrame(array)

# Rename the OID@ field to ObjectID
df_GDD.rename(columns={"OID@": "OBJECTID"}, inplace=True)

# Then we will print the DataFrame to check it
print(df_GDD)


    OBJECTID  nwsli   GDD
0          1  BWNM5  13.0
1          2  WDOM5  11.0
2          3  ARTM5   8.5
3          4  RWGM5   NaN
4          5  ORVM5   NaN
5          6  LUVM5   NaN
6          7  SPGM5   NaN
7          8  ALBM5  10.0
8          9  WNNM5   9.0
9         10  THLM5   8.5
10        11  LMBM5  12.0
11        12  LCHM5  10.5
12        13  AITM5   NaN
13        14  MRAM5   9.5
14        15  MADM5  12.0
15        16  OWAM5  10.0
16        17  LKFM5   9.5
17        18  LEIM5   8.5
18        19  STIM5  10.0
19        20  WAAM5   4.0
20        21  TETM5   NaN
21        22  AGGM5   0.0
22        23  COTM5   NaN
23        24  CRNM5  11.5
24        25  THRM5   8.5
25        26  BRVM5  11.0
26        27  WRIM5   7.0
27        28  ASNM5  13.5
28        29  MMLM5  14.0
29        30  HLLM5   9.0
30        31  MWDM5   NaN
31        32  TOHM5  11.5
32        33  CMBM5   9.5
33        34  ASTM5  13.5
34        35  ALXM5   NaN
35        36  KIMM5  13.0


In [25]:
# Now I will add the sampled interpolated data to the same dataframe
# That will be done by first making individual dataframes for each method
table_paths = [
    "Sample_Idw_ET",
    "Sample_Kriging_ET"
]

dfs = []
for table_path in table_paths:
    array = arcpy.da.TableToNumPyArray(table_path, "*")
    df = pd.DataFrame(array)
    dfs.append(df)

# To check, let's print the dataframes
for i, df in enumerate(dfs):
    print(f"DataFrame {i+1} ({table_paths[i]}):\n{df}\n")


DataFrame 1 (Sample_Idw_ET):
    OBJECTID  Random_Selected_Points         X         Y  IDW_ETPoints_Band_1
0          1                       1 -94.34170  44.73350             0.912743
1          2                       2 -95.10000  43.87360             0.852360
2          3                       3 -96.13330  45.36670             0.880034
3          4                       4 -92.53000  44.57000             1.066189
4          5                       5 -96.42940  45.30890             0.931828
5          6                       6 -96.20220  43.66580             1.007946
6          7                       7 -92.39250  43.69330             0.831419
7          8                       8 -93.30190  43.60640             0.621535
8          9                       9 -94.18730  43.76920             0.791959
9         10                      10 -92.19430  44.28130             0.681674
10        11                      11 -95.32000  44.24000             0.973618
11        12                      1

In [26]:
# This cell will iterate over each data frame in dfs list and merge with df_original
for df in dfs:
    # Perform the merge based on the OBJECTID column
    df_ET = pd.merge(df_ET, df, on='OBJECTID', how='left')

# Check the result
print(df_ET)

    OBJECTID  nwsli        ET  ...       X_y       Y_y  Kriging_ETPoints_Band_1
0          1  BWNM5  0.912751  ... -94.34170  44.73350                 0.912926
1          2  WDOM5  0.852262  ... -95.10000  43.87360                 0.852072
2          3  ARTM5  0.879691  ... -96.13330  45.36670                 0.881274
3          4  RWGM5       NaN  ... -92.53000  44.57000                 1.062485
4          5  ORVM5       NaN  ... -96.42940  45.30890                 0.934326
5          6  LUVM5       NaN  ... -96.20220  43.66580                 1.029243
6          7  SPGM5       NaN  ... -92.39250  43.69330                 0.923166
7          8  ALBM5  0.621449  ... -93.30190  43.60640                 0.622779
8          9  WNNM5  0.791975  ... -94.18730  43.76920                 0.790295
9         10  THLM5  0.681406  ... -92.19430  44.28130                 0.681300
10        11  LMBM5  0.973683  ... -95.32000  44.24000                 0.972392
11        12  LCHM5  1.037180  ... -94.5

In [27]:
# We will perform a little math to calculate the difference in ET between the interpolated points and the samples
df_ET['Idw_ET_difference'] = df_ET['ET'] - df_ET['IDW_ETPoints_Band_1']
df_ET['Kriging_ET_difference'] = df_ET['ET'] - df_ET['Kriging_ETPoints_Band_1']
# Print the DataFrame with the new 
df_ET

,OBJECTID,nwsli,ET,Random_Selected_Points_x,X_x,Y_x,IDW_ETPoints_Band_1,Random_Selected_Points_y,X_y,Y_y,Kriging_ETPoints_Band_1,Idw_ET_difference,Kriging_ET_difference
0,1,BWNM5,0.912751,1,-94.34170,44.73350,0.912743,1,-94.34170,44.73350,0.912926,0.000008,-0.000175
1,2,WDOM5,0.852262,2,-95.10000,43.87360,0.852360,2,-95.10000,43.87360,0.852072,-0.000099,0.000190
2,3,ARTM5,0.879691,3,-96.13330,45.36670,0.880034,3,-96.13330,45.36670,0.881274,-0.000344,-0.001584
3,4,RWGM5,NaN,4,-92.53000,44.57000,1.066189,4,-92.53000,44.57000,1.062485,NaN,NaN
4,5,ORVM5,NaN,5,-96.42940,45.30890,0.931828,5,-96.42940,45.30890,0.934326,NaN,NaN
5,6,LUVM5,NaN,6,-96.20220,43.66580,1.007946,6,-96.20220,43.66580,1.029243,NaN,NaN
6,7,SPGM5,NaN,7,-92.39250,43.69330,0.831419,7,-92.39250,43.69330,0.923166,NaN,NaN
7,8,ALBM5,0.621449,8,-93.30190,43.60640,0.621535,8,-93.30190,43.60640,0.622779,-0.000086,-0.001329
8,9,WNNM5,0.791975,9,-94.18730,43.76920,0.791959,9,-94.18730,43.76920,0.790295,0.000017,0.001680
9,10,THLM5,0.681406,10,-92.19430,44.28130,0.681674,10,-92.19430,44.28130,0.681300,-0.000269,0.000105


In [28]:
#We will then calculate the mean difference 
idw_mean = df_ET['Idw_ET_difference'].mean()
kriging_mean = df_ET['Kriging_ET_difference'].mean()

# Then we will use that to calculate RMSE for each method
idw_rmse = np.sqrt(np.mean(df_ET['Idw_ET_difference']**2))
kriging_rmse = np.sqrt(np.mean(df_ET['Kriging_ET_difference']**2))


# Then we create a dictionary to hold the results
accuracy_results = {
    'Method': ['IDW','Kriging'],
    'Mean': [idw_mean, kriging_mean],
    'RMSE': [idw_rmse, kriging_rmse]
}

# Then we can create DataFrame from the dictionary
accuracy_df_ET = pd.DataFrame(accuracy_results)
print(accuracy_df_ET)

    Method      Mean      RMSE
0      IDW  0.000109  0.000611
1  Kriging  0.000913  0.004048


In [29]:
#This will export the IDW points to a PostGIS database
arcpy.conversion.ExportFeatures(
    in_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Idw_ET_Point",
    out_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\PostgreSQL-34-final_project(postgres).sde\final_project.postgres.idw_et_point"
)
print("Export complete")

Export complete
